## Javier Mombiela
## Carnet: 20067
## 12 de abril 2024

### Proyecto 2: “Entrenamiento Incremental en Modelos de Deep Learning y Machine Learning

Importando librerias

In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

Cargar el dataset

In [5]:
df = pd.read_csv('fraud_feature_engineering_example.csv')

df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,4.97,4.97,0.0,0.0,True
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,107.23,107.23,0.0,0.0,True
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,220.11,220.11,0.0,0.0,True
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,45.00,45.00,0.0,0.0,True
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,41.96,41.96,0.0,0.0,True


#### Preprocesamiento

Dividir datos

In [6]:
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Dimensiones de X: (1852394, 26)
Dimensiones de y: (1852394,)


### Implementación Practica de los modelos

#### Random Forest

In [7]:
# Entrenar el modelo inicial de Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Aplicar entrenamiento incremental para Random Forest si es necesario
rf_model.fit(X_new, y_new)

# Calcular las métricas para Random Forest
y_dev_pred_rf = rf_model.predict_proba(X_dev)[:, 1]  # Probabilidad de ser fraude
roc_auc_rf = roc_auc_score(y_dev, y_dev_pred_rf)
precision_rf = precision_score(y_dev, rf_model.predict(X_dev))
recall_rf = recall_score(y_dev, rf_model.predict(X_dev))
f1_rf = f1_score(y_dev, rf_model.predict(X_dev))

print("Random Forest Metrics:")
print("ROC-AUC:", roc_auc_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1-Score:", f1_rf)

ValueError: could not convert string to float: '2020-07-06 18:23:19'